In [1]:
import copy
import time
import numpy as np
import math
import itertools
from itertools import permutations
import open3d as o3d
#import torch
import pandas as pd
from origin_apf_test2 import apf_env,distanceCost,Traditional_Apf_Method,cal_angle
#from Model22 import Actor_MultiModal2
#from test import pre_process
import os
import random
import matplotlib.pyplot as plt
from gen_env import generate_columns,generate_spheres,generate_cubes,gen_sign

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
forest=o3d.io.read_point_cloud("./pcd/rollout_21-02-07_03-14-08_pointcloud-unity.ply")
pts = [p for p in np.asarray(forest.points) if p[2]<3]
forest_ = o3d.geometry.PointCloud(
o3d.utility.Vector3dVector( np.asarray(forest.points))
)
o3d.visualization.draw_geometries([forest_
    ,
    gen_sign(np.array([10,20])),gen_sign(np.array([20,10]))
])

In [6]:
t_x = np.linspace(-10,20,300)
t_y = np.linspace(-10,10,200)
t_z = np.linspace(-0.4,0.4,6)
t = []
for x in t_x:
    for y in t_y:
        for z in t_z:
            t.append([x,y,z])
t = np.array(t)
terrain = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(t))
terrain.paint_uniform_color(np.array([20,20,10])/255.)

PointCloud with 360000 points.

In [16]:
wall = [generate_cubes(0.5,np.array([i,1.]),5) for i in range(3)]+[generate_cubes(0.5,np.array([i,0.]),5) for i in range(3)]

wall_pt = [np.asarray(w.points) for w in wall]
wall_pt = np.asarray(wall_pt).reshape(-1,3)
wall_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(wall_pt))
# o3d.visualization.draw_geometries([*wall,terrain])
# o3d.visualization.draw_geometries([wall_pcd])

In [21]:
wall2 = [generate_cubes(0.5,np.array([i,1.]),5) for i in range(3) ]+\
        [generate_cubes(0.2,np.array([-0.3,1.7]),5),
         generate_cubes(0.2,np.array([2.3,2.1]),5),
         generate_cubes(0.2,np.array([-0.3,2.1]),5),
         generate_cubes(0.2,np.array([2.3,1.7]),5),
         ]
o=o3d.geometry.PointCloud()
for w in wall2:
    o+=w
o+=terrain
o3d.io.write_point_cloud("./pcd/s2.ply",o)

True

In [7]:
o3d.visualization.draw_geometries(
    [
        o3d.io.read_point_cloud("./pcd/s2.ply"),gen_sign(np.array([1.2,2])),
        gen_sign(np.array([-2,0.5])),
        gen_sign(np.array([2,-3]))
    ]
)

In [99]:
tree = o3d.geometry.KDTreeFlann(wall_pcd)
ret=tree.search_knn_vector_3d(np.array([0,0,6]),10)
sum(ret[2])

3.6274999999999964

In [102]:

def calc_bending(waypts):
    ret = 0
    for i in range(len(waypts) - 2):
        v1 = waypts[i + 1][:2] - waypts[i][:2]
        v2 = waypts[i + 2][:2] - waypts[i + 1][:2]
        ret += cal_angle(v1, v2) / 60
    ret = ret / (len(waypts) - 2)
    # assert ret < 1.4,(ret,waypts)
    return ret

def calc_dist(waypts):
    ret = 0
    line_dist = distanceCost(waypts[0], waypts[-1])
    for i in range(len(waypts) - 1):
        ret += distanceCost(waypts[i + 1], waypts[i])
    return ret / line_dist
def calc_safety(path,pcd):
    tree = o3d.geometry.KDTreeFlann(pcd)
    d = 0
    for pt in path[::5]:
        ret = tree.search_knn_vector_3d(pt,50)
        d+=sum(ret[2])

    return d/(path.shape[0]/5)
def get_reward(path:np.ndarray):
    # r1 theta and bending
    # r2 safe d
    r1 = max(calc_bending(path) * -2.5, -5)
    r2 = calc_dist(path) * -1
    #r3 = calc_safety(pos) * -5
    r3 =0
    return (r1,r2,r3)

In [53]:
path1 = np.hstack((np.linspace(0,10,100).reshape(-1,1),np.array([0]*100).reshape(-1,1),np.array([1]*100).reshape(-1,1)))
x =  np.linspace(0,10,100)
for i in range(8,11):
    exec(f'f{i}= lambda x: math.sin(2*math.pi/10*x)*0.5*(i+1)')
    exec(f'path{i}=np.hstack((x.reshape(-1,1),np.array([f{i}(e) for e in x]).reshape(-1,1),np.array([1]*100).reshape(-1,1)))')
l = [exec(f'path{i}') for i in range(8)]
for i in range(8):
    np.savetxt(f"./res/res3/path{i}.csv",eval(f'path{i}'),delimiter=',')

In [55]:
r = []

for i in range(8,15):
    v = eval(f'path{i}')
    r.append((calc_dist(v),calc_bending(v)))
r2 = np.array([sum(x) for x in r])


In [103]:
r = []
for i in range(8,15):
    v = eval(f'path{i}')
    r.append(calc_safety(v,wall_pcd))


In [5]:
for i in range(8,15):
    exec(f'f{i}= lambda x: x+np.array([0,-1.5-0.3*(i-7),1])')
    exec(f'path{i}= f{i}(path1)')
l = [exec(f'path{i}') for i in range(8)]
for i in range(8,15):
    np.savetxt(f"./res/res3/path{i}.csv",eval(f'path{i}'),delimiter=',')

NameError: name 'path1' is not defined

In [4]:
path_list2 = [path8,path9,path10,path11,path12,path13,path14]
path_list2_pcd = [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(ele)) for ele in path_list2]

NameError: name 'path8' is not defined

In [94]:
color_ = [np.array([247,208,59])/255.,
          np.array([250,157,58])/255.,
          np.array([236,120,83])/255.,
          np.array([215,86,107])/255.,
          np.array([188,54,132])/255.,
          np.array([155,23,159])/255.,
          np.array([114,0,169])/255.,]
for x,c in zip(path_list2_pcd,color_):
    x.paint_uniform_color(c)
o3d.visualization.draw_geometries([*path_list2_pcd,*wall,terrain])

In [104]:
0.01*(r[-1]-r[0])

4.862287869187006

In [58]:
np.savetxt("./res/res3/rew.csv",np.linspace(r2[0],r2[-1],500),delimiter=',')
6.74-(x-1)/256*4.862287869187006

In [11]:
path13=pd.read_csv("./res/path13.csv",sep=',',header=None).to_numpy()
path13=path13[::2]
o3d.visualization.draw_geometries([
    o3d.io.read_point_cloud(r"./pcd/t3.ply"),
    #o3d.io.read_point_cloud(r"./res/r13.ply")
    o3d.geometry.PointCloud(o3d.utility.Vector3dVector(np.asarray(path13)))
])